In [3]:
%pip install pandas plotly dash dash-core-components dash-html-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 62.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

try:

    df = pd.read_csv('/content/social_network_analytics_100_students_1000_rows.csv')
    df['post_time'] = pd.to_datetime(df['post_time'])
    df['login_time'] = pd.to_datetime(df['login_time'])
    df['post_hour'] = df['post_time'].dt.hour

except FileNotFoundError:
    print("Error: 'social_network_analytics_100_students_1000_rows.csv' not found.")
    print("Please make sure the CSV file is in the same directory as this script.")

    df = pd.DataFrame({
        'school': ['N/A'], 'class': ['N/A'], 'gender': ['N/A'],
        'engagement': [0], 'likes': [0], 'comments': [0], 'shares': [0],
        'post_hour': [0], 'student_id': [0]
    })



app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.title = "Student Social Analytics"


app.layout = html.Div(children=[

    html.Div(className='row', children=[
        html.H1(
            children='Social Network Analytics Dashboard',
            style={'textAlign': 'center', 'color': '#450'}
        ),
        html.Div(
            children='An interactive dashboard to explore student engagement and activity.',
            style={'textAlign': 'center', 'color': '#466'}
        )
    ]),

    html.Hr(),


    html.Div(className='row', children=[
        html.Div(className='three columns', children=[
            html.Label('Filter by School:'),
            dcc.Dropdown(
                id='school-filter',
                options=[{'label': i, 'value': i} for i in df['school'].unique()],
                value=df['school'].unique().tolist(), # Default value
                multi=True,
                placeholder="Select school(s)"
            )
        ]),
        html.Div(className='three columns', children=[
            html.Label('Filter by Class:'),
            dcc.Dropdown(
                id='class-filter',
                options=[{'label': i, 'value': i} for i in sorted(df['class'].unique())],
                value=sorted(df['class'].unique()),
                multi=True,
                placeholder="Select class(es)"
            )
        ]),
        html.Div(className='three columns', children=[
            html.Label('Filter by Gender:'),
            dcc.Dropdown(
                id='gender-filter',
                options=[{'label': i, 'value': i} for i in df['gender'].unique()],
                value=df['gender'].unique().tolist(),
                multi=True,
                placeholder="Select gender(s)"
            )
        ]),
    ], style={'padding': '20px 0'}),


    html.Div(className='row', children=[

        html.Div(className='six columns', children=[
            dcc.Graph(id='engagement-metrics-bar'),
            dcc.Graph(id='gender-distribution-pie')
        ]),

        html.Div(className='six columns', children=[
            dcc.Graph(id='likes-vs-comments-scatter'),
            dcc.Graph(id='hourly-activity-histogram')
        ])
    ])
])


@app.callback(
    [Output('engagement-metrics-bar', 'figure'),
     Output('likes-vs-comments-scatter', 'figure'),
     Output('gender-distribution-pie', 'figure'),
     Output('hourly-activity-histogram', 'figure')],
    [Input('school-filter', 'value'),
     Input('class-filter', 'value'),
     Input('gender-filter', 'value')]
)
def update_graphs(selected_schools, selected_classes, selected_genders):
    if not isinstance(selected_schools, list): selected_schools = [selected_schools]
    if not isinstance(selected_classes, list): selected_classes = [selected_classes]
    if not isinstance(selected_genders, list): selected_genders = [selected_genders]


    filtered_df = df[
        df['school'].isin(selected_schools) &
        df['class'].isin(selected_classes) &
        df['gender'].isin(selected_genders)
    ]


    if not filtered_df.empty:
        school_engagement = filtered_df.groupby('school')[['likes', 'comments', 'shares']].mean().reset_index()
        bar_fig = px.bar(
            school_engagement,
            x='school',
            y=['likes', 'comments', 'shares'],
            title='Average Engagement Metrics by School',
            labels={'value': 'Average Count', 'school': 'School'},
            barmode='group'
        )
    else:
        bar_fig = px.bar(title='No data for selected filters')



    scatter_fig = px.scatter(
        filtered_df,
        x='comments',
        y='likes',
        color='school',
        hover_name='name',
        title='Likes vs. Comments per Post',
        labels={'comments': 'Number of Comments', 'likes': 'Number of Likes'}
    )

    if not filtered_df.empty:
        gender_counts = filtered_df.groupby('gender').size().reset_index(name='count')
        pie_fig = px.pie(
            gender_counts,
            values='count',
            names='gender',
            title='Gender Distribution of Students',
            color_discrete_sequence=px.colors.sequential.RdBu
        )
    else:
        pie_fig = px.pie(title='No data for selected filters')


    hist_fig = px.histogram(
        filtered_df,
        x='post_hour',
        nbins=24,
        title='Post Activity by Hour of Day',
        labels={'post_hour': 'Hour of Day (24h format)', 'count': 'Number of Posts'}
    )
    hist_fig.update_layout(bargap=0.1)

    return bar_fig, scatter_fig, pie_fig, hist_fig

if __name__ == '__main__':
    app.run(debug=True)

/tmp/ipython-input-1627534169.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipython-input-1627534169.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<IPython.core.display.Javascript object>